In [ ]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import pickle

# Load models, vectorizer, and label encoders
with open('capital_model_SVM_withoutGS.pkl', 'rb') as f:
    capital_model = pickle.load(f)

with open('sentiment_model_LR_withoutGS.pkl', 'rb') as f:
    sentiment_model = pickle.load(f)

with open('tfidf_vectorizer_SVM_LR_withoutGS.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

with open('label_encoder_capital_SVM_LR_withoutGS.pkl', 'rb') as f:
    label_encoder_capital = pickle.load(f)

with open('label_encoder_sentiment_SVM_LR_withoutGS.pkl', 'rb') as f:
    label_encoder_sentiment = pickle.load(f)


# Define stop words
stop_words = set(stopwords.words('english'))

# Define text preprocessing functions
def remove_unnecessary_characters(text):
    text = re.sub(r'<.*?>', '', str(text))
    text = re.sub(r'[^a-zA-Z0-9\s]', '', str(text))
    text = re.sub(r'\s+', ' ', str(text)).strip()
    return text

def tokenize_text(text):
    try:
        text = str(text)
        tokens = word_tokenize(text)
        return tokens
    except Exception as e:
        print(f"Error tokenizing text: {e}")
        return []

def predict_pipeline(text):
    clean_text = remove_unnecessary_characters(text)
    tokens = tokenize_text(clean_text)
    text_without_stopwords = ' '.join([word for word in tokens if word.lower() not in stop_words])
    text_tfidf = tfidf_vectorizer.transform([text_without_stopwords])
    
    capital_prediction_encoded = capital_model.predict(text_tfidf)
    sentiment_prediction_encoded = sentiment_model.predict(text_tfidf)
    
    capital_prediction = label_encoder_capital.inverse_transform(capital_prediction_encoded)[0]
    sentiment_prediction = label_encoder_sentiment.inverse_transform(sentiment_prediction_encoded)[0]
    
    return capital_prediction, sentiment_prediction

# Test the prediction function
test_sentence = "Leaders' focus on achieving excellence motivated the team to perform their best. This is a core aspect of quality leadership. 🔝"
capital, sentiment = predict_pipeline(test_sentence)
print(f"Predicted Capital: {capital}, Predicted Sentiment: {sentiment}")


In [ ]:
test_sentence = "✨ They found meaning and purpose in life through their spirituality."
capital, sentiment = predict_pipeline(test_sentence)
print(f"Predicted Capital: {capital}, Predicted Sentiment: {sentiment}")


In [ ]:
import tkinter as tk
from tkinter import simpledialog, messagebox

In [ ]:
def predict():
    text = entry.get()
    if not text:
        messagebox.showwarning("Input Error", "Please enter a sentence.")
        return
    predicted_capital, predicted_sentiment = predict_pipeline(text)

    result_text = f"Predicted Capital: {predicted_capital}\nPredicted Sentiment: {predicted_sentiment}"
    result_window = tk.Toplevel(root)
    result_window.title("Prediction Result")

    result_label = tk.Label(result_window, text=result_text, font=("Helvetica", 14), padx=20, pady=20)
    result_label.pack()

    close_button = tk.Button(result_window, text="Close", command=result_window.destroy, font=("Helvetica", 12), padx=10, pady=5)
    close_button.pack(pady=10)

root = tk.Tk()
root.title("Capital and Sentiment Prediction")
# Set window size and position
root.geometry("500x250+300+200")

label = tk.Label(root, text="Enter a sentence:", font=("Helvetica", 14))
label.pack(pady=20)

entry = tk.Entry(root, width=50, font=("Helvetica", 12))
entry.pack(pady=10)

predict_button = tk.Button(root, text="Predict", command=predict, font=("Helvetica", 12), padx=10, pady=5)
predict_button.pack(pady=20)

root.mainloop()

# Thank You!